## Install dependencies

In [ ]:
from IPython import display
!pip install wandb --q
display.clear_output()

In [ ]:
import cv2
import numpy as np
import pandas as pd
import os
import glob
import time

import random
from tqdm import tqdm
from collections import Counter

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from mlxtend.plotting import plot_confusion_matrix

import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Activation, BatchNormalization, Dense, Dropout, Flatten, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras import models
from tensorflow.keras.utils import load_img, img_to_array
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import confusion_matrix

In [ ]:
train_df = pd.read_csv("/kaggle/input/challenges-in-representation-learning-facial-expression-recognition-challenge/train.csv")
test_df = pd.read_csv("/kaggle/input/challenges-in-representation-learning-facial-expression-recognition-challenge/train.csv")
data = pd.read_csv('/kaggle/input/challenges-in-representation-learning-facial-expression-recognition-challenge/icml_face_data.csv')

In [ ]:
emotions = {0: 'Angry', 1: 'Disgust', 2: 'Fear', 3: 'Happy', 4: 'Sad', 5: 'Surprise', 6: 'Neutral'}

In [ ]:
data[' Usage'].value_counts()

In [ ]:
train_df.head(3)

In [ ]:
# (0=Angry, 1=Disgust, 2=Fear, 3=Happy, 4=Sad, 5=Surprise, 6=Neutral).
train_df['emotion'].value_counts()

In [ ]:
test_df['emotion'].value_counts()

## Helper functions

In [ ]:
def load_data(data):
    image_array = np.zeros(shape=(len(data), 48, 48)) #(48, 48)
    image_label = np.array(list(map(int, data['emotion']))) # [0, 0, 1, 2, ...]
    
    for count, row in enumerate(data.index): # from 0 to 35887
        image = np.fromstring(data.loc[row, ' pixels'], dtype=int, sep=' ') # [ 70  80  82 ... 106 109  82], np array, shape (2304,)
        image = np.reshape(image, (48, 48)) # shape (48, 48)
        image_array[count] = image
        
    return image_array, image_label


def plot_image_and_emotion(test_image_array, test_image_label, pred_test_labels, image_number):
    """ Function to plot the image and compare the prediction results with the label """
    
    fig, axs = plt.subplots(1, 2, figsize=(12, 6), sharey=False)
    
    bar_label = emotions.values()
    
    axs[0].imshow(test_image_array[image_number], 'gray')
    axs[0].set_title(emotions[test_image_label[image_number]])
    
    axs[1].bar(bar_label, pred_test_labels[image_number], color='orange', alpha=0.7)
    axs[1].grid()
    
    plt.show()
    
    
def plot_compare_distributions(array1, array2, title1='', title2=''):
    df_array1 = pd.DataFrame()
    df_array2 = pd.DataFrame()
    df_array1['emotion'] = array1.argmax(axis=1)
    df_array2['emotion'] = array2.argmax(axis=1)
    
    fig, axs = plt.subplots(1, 2, figsize=(12, 6), sharey=False)
    x = emotions.values()
    
    y = df_array1['emotion'].value_counts()
    keys_missed = list(set(emotions.keys()).difference(set(y.keys())))
    for key_missed in keys_missed:
        y[key_missed] = 0
    axs[0].bar(x, y.sort_index(), color='orange')
    axs[0].set_title(title1)
    axs[0].grid()
    
    y = df_array2['emotion'].value_counts()
    keys_missed = list(set(emotions.keys()).difference(set(y.keys())))
    for key_missed in keys_missed:
        y[key_missed] = 0
    axs[1].bar(x, y.sort_index())
    axs[1].set_title(title2)
    axs[1].grid()
    
    plt.show()

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

def callbackFunction(modelName):
    checkpoint = ModelCheckpoint(f"Checkpoints/{modelName}.h5", monitor = "val_accuracy", save_best_only= True, mode = "auto", verbose = 1)
    early_stopping = EarlyStopping(monitor= "val_accuracy", patience=20, verbose = 1)
    callbacks = [early_stopping, checkpoint]
    return callbacks

## Integrate with WandB

In [ ]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("uow-wandb")

In [ ]:
import wandb
from wandb.keras import WandbCallback 
wandb.login(key=secret_value_0)

## Split and preprocess data

In [ ]:
train_image_array, train_image_label = load_data(data[data[' Usage']=='Training'])
val_image_array, val_image_label = load_data(data[data[' Usage']=='PrivateTest'])
test_image_array, test_image_label = load_data(data[data[' Usage']=='PublicTest'])

In [ ]:
train_images = train_image_array.reshape((train_image_array.shape[0], 48, 48, 1))
train_images = train_images.astype('float32')/255 # normalize images

val_images = val_image_array.reshape((val_image_array.shape[0], 48, 48, 1))
val_images = val_images.astype('float32')/255 # normalize images

test_images = test_image_array.reshape((test_image_array.shape[0], 48, 48, 1))
test_images = test_images.astype('float32')/255 # normalize images

In [ ]:
train_labels = to_categorical(train_image_label)
val_labels = to_categorical(val_image_label)
test_labels = to_categorical(test_image_label)

In [ ]:
class_weight = dict(zip(range(0, 7), (((data[data[' Usage']=='Training']['emotion'].value_counts()).sort_index())/len(data[data[' Usage']=='Training']['emotion'])).tolist()))
class_weight

In [ ]:
dataset_ratio = f"{len(train_images)}/{len(val_images)}/{len(test_images)}"

In [ ]:
config = dict (
              num_classes=len(emotions),
              input_shape=(48, 48, 1),
              batch_size = 64,
              epochs= 50,
              loss="categorical_crossentropy",
              optimizer="adam",
              learning_rate=1e-3,
              augmentation=None,
              dataset_ratio=dataset_ratio)

## Model 1

In [ ]:
model1 = models.Sequential()

model1.add(Conv2D(64, (3, 3), padding='same', activation='relu', input_shape=(48, 48, 1)))
model1.add(Dropout(0.25))

model1.add(MaxPool2D((2, 2)))
model1.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model1.add(Dropout(0.25))

model1.add(MaxPool2D((2, 2)))
model1.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
model1.add(Dropout(0.25))

model1.add(MaxPool2D((2, 2)))
model1.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
model1.add(Dropout(0.25))

model1.add(MaxPool2D((2, 2)))
model1.add(Flatten())
model1.add(Dense(256, activation='relu'))
model1.add(Dense(128, activation='relu'))
model1.add(Dropout(0.25))

model1.add(Dense(7, activation='softmax'))

In [ ]:
config['model_summary'] = model1.summary()
wandb.init(name='exp-1.1', project="FER(UOW)",  entity="brian-uow", config=config)

## Compile Model

In [ ]:
model1.compile(optimizer=Adam(learning_rate=1e-3), loss=config['loss'], metrics=['accuracy'])

In [ ]:
callbacks = [callbackFunction('exp-1'), WandbCallback()]

In [ ]:
history1 = model1.fit(train_images, train_labels,
                    validation_data=(val_images, val_labels),
                    callbacks=callbacks,
                    class_weight = class_weight,
                    epochs=config['epochs'],
                    batch_size=config['batch_size'])
wandb.finish()

In [ ]:
pred_test_labels1 = model1.predict(test_images)

## Plotting

In [ ]:
loss = history1.history['loss']
loss_val = history1.history['val_loss']
epochs = range(1, len(loss)+1)
plt.plot(epochs, loss, 'bo', label='loss_train')
plt.plot(epochs, loss_val, 'b', label='loss_val')
plt.title('value of the loss function')
plt.xlabel('epochs')
plt.ylabel('value of the loss function')
plt.legend()
plt.grid()
plt.show()

In [ ]:
acc = history1.history['accuracy']
acc_val = history1.history['val_accuracy']
epochs = range(1, len(loss)+1)
plt.plot(epochs, acc, 'bo', label='accuracy_train')
plt.plot(epochs, acc_val, 'b', label='accuracy_val')
plt.title('accuracy')
plt.xlabel('epochs')
plt.ylabel('value of accuracy')
plt.legend()
plt.grid()
plt.show()

## Results

In [ ]:
plot_image_and_emotion(test_image_array, test_image_label, pred_test_labels1, image_number=19)

In [ ]:
plot_image_and_emotion(test_image_array, test_image_label, pred_test_labels1, 119)

In [ ]:
plot_compare_distributions(test_labels, pred_test_labels1, title1='Test Labels', title2='Predicted Labels')

## Confusion Matrix

In [ ]:
df_compare = pd.DataFrame()
df_compare['real'] = test_labels.argmax(axis=1)
df_compare['pred'] = pred_test_labels1.argmax(axis=1)
df_compare['wrong'] = np.where(df_compare['real']!=df_compare['pred'], 1, 0)

In [ ]:
conf_mat = confusion_matrix(test_labels.argmax(axis=1), pred_test_labels1.argmax(axis=1))

fig, ax = plot_confusion_matrix(conf_mat=conf_mat,
                                show_normed=True,
                                show_absolute=False,
                                class_names=emotions.values(),
                                figsize=(8, 8))
fig.show()